In [1]:
import pandas as pd
import numpy as np
import random
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from statistics import mean

from sklearn.model_selection import GridSearchCV, \
    RandomizedSearchCV  # Hyperparameter tuning - GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, \
    KFold  # Cross-validation - StratifiedKFold, RepeatedStratifiedKFold

from sklearn.ensemble import RandomForestClassifier  # Classifier - Random Forest
from sklearn.linear_model import LogisticRegression  # Classifier - Logistic Regression
from sklearn.tree import DecisionTreeClassifier  # Classifier - Decision Tree

from sklearn.metrics import roc_auc_score  # Evaluation metric - AUC
from sklearn.metrics import f1_score  # Evaluation metric - F1 score
from sklearn.metrics import accuracy_score  # Evaluation metric - Accuracy

In [2]:
# Load data
data = pd.read_table('Train_call.txt')  # Shows the data with samples as rows, so need to transpose
data = data.T  # Transposes data, so that samples are now rows.
data_target = pd.read_table('Train_clinical.txt')  # Gives sample with associated subgroup

# Extract predictor and target data
target = data_target.loc[:,
         "Subgroup"]  # Isolates the subgroups from samples. We need to convert the subgroups into 0, 1, 2
new_data_unlabeled = data.iloc[4:, :]  # This is the complete cleaned up dataset

# We also need to convert the subgroups into 0, 1, 2
for i in range(len(target)):
    if target[i] == "HER2+":
        target[i] = 0
    elif target[i] == "HR+":
        target[i] = 1
    elif target[i] == "Triple Neg":
        target[i] = 2

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(new_data_unlabeled, target, test_size=0.2,
                                                    random_state=42)
y_train = y_train.astype(int)  # For some reason the numbers are read as strings, so convert to integers

# Hyperparameter grid
rf_hp_grid = param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}
dt_hp_grid = {'max_depth': [None, 5, 20]}

# CV technique for outer and inner folds
outer_cv = StratifiedKFold(n_splits=4)
inner_cv = StratifiedKFold(n_splits=6)

# -----------
# Plan:
# 1) We already have a train/test split
# 2) Cross validation: we split the training data into 4-folds: train + validation
# 3) For each fold, we apply feature  selection to reduce the dataset
# 4) For each fold, we apply 6-CV for hyperparameter tuning
# 5) For each fold, we test the optimal models

# Step 1) Is already done
# Step 2) We split the data into 4 folds (outer folds)
y_train.reset_index(drop=True, inplace=True)  # We need to reset the indices of y_train so we can apply CV split
dt_fold_performance_lst = []  # For df and rf we will append their performances to these lists and then take the mean of these
rf_fold_performance_lst = []

dt_fold_hp_lst = []  # For df and rf we will append the best hyperparameters to these lists
rf_fold_hp_lst = []

In [3]:
for i, (train_index, test_index) in enumerate(outer_cv.split(X_train, y_train)):
    print(f"We are currently in Outer fold {i + 1}")
    X_train_fold = X_train.iloc[train_index,
                   :]  # train_index is a list of indices, but we can pass lists of indices in np
    y_train_fold = y_train[train_index]
    X_validate_fold = X_train.iloc[test_index, :]
    y_validate_fold = y_train[test_index]

    # Step 3) Feature selection

    # Combining X_train and y_train again in one dataframe
    y_train_df = pd.DataFrame(y_train_fold)
    train_data = X_train_fold.copy()
    train_data['Subgroup'] = list(y_train_df.iloc[:, 0])

    # Perform Chi-Squared test for each feature (column)
    chi2_results = []
    target_variable = 'Subgroup'

    # For each feature, find correlation with target variable
    for feature in train_data.columns:
        if feature != target_variable:  # Check if the feature is categorical
            contingency_table = pd.crosstab(train_data[feature], train_data[target_variable])
            results = chi2_contingency(contingency_table)
            chi2_results.append({'Feature': feature, 'Chi-Squared': results[0], 'p-value': results[1]})

    df_chi = pd.DataFrame(chi2_results)  # Save the results into a dataframe

    # Filter out the features with insignificant target correlations
    significant = []
    for j in range(0, len(df_chi['p-value'])):
        if df_chi.iloc[j, 2] <= 0.05:  # when p-value is smaller than or equal to critical value
            significant.append(True)
        else:
            significant.append(False)

    # Append whether feature is significant (True/False) to chi dataframe
    df_chi['significant'] = significant

    # Select the significant features
    significant_features = df_chi[df_chi['significant']]
    features = pd.DataFrame(significant_features['Feature'])
    features_list = list(features['Feature'])

    # Make for each feature an empty set
    clusters_list = [set() for i in range(len(features_list) + 1)]
    k = 0

    # Go through each significant feature
    for j in range(len(features_list) - 1):
        # Retrieve feature information
        feature_j_data = X_train.iloc[:, features_list[j]]
        feature_neighbour_data = X_train.iloc[:, features_list[j + 1]]
        current_cluster = clusters_list[k]

        # If cluster is empty, add current feature to it
        if len(current_cluster) == 0:
            current_cluster.add(features_list[j])

        # If neighboring significant feature has high correlation, add to current cluster. Else, create new cluster
        if abs(feature_j_data.corr(feature_neighbour_data)) > 0.8:
            # print("Feature",features_list[j],"and feature",features_list[j+1], "are correlated")
            current_cluster.add(features_list[j + 1])
        else:
            # print("Feature",features_list[j],"and feature", features_list[j+1], "are not correlated")
            k += 1  # we need to go to a new cluster

            # If the final feature does not correlate, we still need to create its own cluster
            if features_list[j + 1] == features_list[-1]:
                current_cluster = clusters_list[k]
                current_cluster.add(features_list[j + 1])

    # Remove all the empty clusters from clusters_list
    clusters_list = [cluster for cluster in clusters_list if cluster != set()]

    # from each cluster we randomly pick one feature
    indep_features_list = []
    for cluster in clusters_list:
        feature_random = random.choice(list(cluster))
        indep_features_list.append(feature_random)

    print('This fold has', len(indep_features_list), 'independent features')
    # select the X_train_fold data for only independent features
    r_X_train_fold = X_train_fold.iloc[:, indep_features_list]
    r_X_validate_fold = X_validate_fold.iloc[:, indep_features_list]

    # Step 4) Apply 5CV Grid search to each X_train_fold

    # Define classifiers
    rf_classifier = RandomForestClassifier(random_state=42)
    dt_classifier = DecisionTreeClassifier(random_state=42)

    # Define grid search object for all classifiers
    # USE ACCURACY AS A PLACEHOLDER MEASURE!!! (to be removed)
    rf_grid_search = GridSearchCV(estimator=rf_classifier, param_grid=rf_hp_grid, cv=inner_cv, scoring='accuracy',
                                  verbose=1)
    dt_grid_search = GridSearchCV(estimator=dt_classifier, param_grid=dt_hp_grid, cv=inner_cv, scoring='accuracy',
                                  verbose=1)

    # Run grid search for all classifiers on the training data of this current fold
    rf_grid_search.fit(r_X_train_fold, y_train_fold)
    dt_grid_search.fit(r_X_train_fold, y_train_fold)

    # Extract the most important parameter and the corresponding score
    rf_best_hp = rf_grid_search.best_params_
    rf_best_score = rf_grid_search.best_score_

    dt_best_hp = dt_grid_search.best_params_
    dt_best_score = dt_grid_search.best_score_

    # store the best hyperparameters in the dictionaries
    rf_fold_hp_lst.append([rf_best_hp, rf_best_score])
    dt_fold_hp_lst.append([dt_best_hp, dt_best_score])

    # Define new models with the optimal hyperparameters
    best_rf_classifier = rf_grid_search.best_estimator_
    best_dt_classifier = dt_grid_search.best_estimator_

    # Step 5) Now that we have the best models for this fold, we can test the performance on X_train_fold
    rf_y_predict_fold = best_rf_classifier.predict(r_X_validate_fold)
    dt_y_predict_fold = best_dt_classifier.predict(r_X_validate_fold)

    # Retrieve the accuracy score
    rf_accuracy = accuracy_score(rf_y_predict_fold, y_validate_fold)
    dt_accuracy = accuracy_score(dt_y_predict_fold, y_validate_fold)

    print("\nRandom forest best hp:", rf_best_hp, "Score on validation fold:", rf_accuracy)
    print("Decision tree best hp:", dt_best_hp, "Score on validation fold:", dt_accuracy)

    # Append performance to list
    rf_fold_performance_lst.append(rf_accuracy)
    dt_fold_performance_lst.append(dt_accuracy)

    print("Performance of random forest per fold", rf_fold_performance_lst, "Average performance:",
          mean(rf_fold_performance_lst))
    print("Performance of decision tree per fold", dt_fold_performance_lst, "Average performance:",
          mean(dt_fold_performance_lst))

    # -------------------------------------------------
    # Feature selection for this current fold:
    # Print cluster (to map features to)
    print("\nThese are the clusters:", clusters_list)

    # Retrieve feature importances and names of features
    importance = best_rf_classifier.feature_importances_
    names_features = r_X_train_fold.columns

    # plot feature importance
    forest_importances = pd.Series(importance, index=names_features)
    sort_forest_importances = forest_importances.sort_values(ascending=False)
    top_forest_importances = sort_forest_importances[:10]

    print("\nFor this fold, these are the top 10 important features:")
    print(top_forest_importances)

    # ----------------------------

We are currently in Outer fold 1
This fold has 94 independent features
Fitting 6 folds for each of 216 candidates, totalling 1296 fits
Fitting 6 folds for each of 3 candidates, totalling 18 fits

Random forest best hp: {'bootstrap': False, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 100} Score on validation fold: 0.65
Decision tree best hp: {'max_depth': 5} Score on validation fold: 0.8
Performance of random forest per fold [0.65] Average performance: 0.65
Performance of decision tree per fold [0.8] Average performance: 0.8

These are the clusters: [{2}, {3}, {5}, {38, 39}, {57, 58}, {101}, {230}, {271}, {305, 308, 302, 303}, {371}, {375}, {387, 388}, {416}, {480, 481, 482, 483, 484, 463, 465, 466, 471, 472, 473, 474, 475}, {485}, {486}, {488, 489, 487}, {491, 492, 493, 495}, {596}, {608, 609}, {610, 611}, {612, 613}, {616, 618, 619, 615}, {620, 621, 622, 623}, {665, 666, 667, 668, 669, 670, 671}, {672, 673, 674, 675, 676, 679